# Assignment 3: MapReduce & Web Graph

## General guidelines

This notebook contains considerable amount of code to help you complete this assignment. Your task is to implement any missing parts of the code and answer any questions (if exist) within this notebook. This will require understanding the existing code, may require reading about packages being used, reading additional resources, and maybe even going over your notes from class :😱

**Evaluation and auto-grading**: Your submissions will be evaluated using both automatic and manual grading. Code parts for implementation are marked with a comment `# YOUR CODE HERE`, and usually followed by cell(s) containing automatic tests that evaluate the correctness of your answer. Staff will allow your notebook to **execute from start to finish for no more than 90 seconds**, then manually assess your submission. Any automatic tests that did not run due to your notebook timing out **will automatically receive 0 points**. The execution time excludes initial data download, which will already exist in the testing environment. The staff reserves the right to **modify any grade provided by the auto-grader** as well as to **execute additional tests not provided to you**. It is also important to note that **auto-graded cells only result in full or no credit**. In other words, you must pass all tests implemented in a test cell in order to get the credit for it, and passing some, but not all, of the tests in a test cell will not earn you any points for that cell. 

**Submission**: Upload a zip file **with your ID as the file name** containing the colab notebook and the GCP notebook to the assignment on Moodle. The names of the files should be  in the format:  012345678_colab.ipynb and 012345678_gcp.ipynb.

# Tasks

In this assignment, we are going to use MapReduce in pySpark to scale up index creation. **The assignment consists of two parts, one in Google Colab and one in GCP**. In the first part (Colab), we will learn how to use MapReduce in PySpark, create an inverted index, and calculate PageRank. In the second part, we will use GCP to scale up index creation for the entire English Wikipedia. 

You will learn how to work with PySpark, write MapReduce code that executes on a cluster of machines, and experience the power of parallel processing. **All tasks must be completed through the use of PySpark's MapReduce**.

To save you (and us) time, both in this assignment and in the final project, we are providing you with preprocessed files for the entire English wikipedia dump. Preprocessing was done, similar to Assignment #2, to extract article information from the XML, then remove the WikiMedia markup. The complete preprocessing code for one wiki dump can be seen [here](https://colab.research.google.com/drive/1xXc0YPU4BVd8CAjVNgICimW_d-Z7wryu?usp=sharing).


**By the end of this assignment, you will have:**

In Colab (65 Points)

1. (20 Points) Processed articles' body to produce posting lists (using PySpark and MapReduce).
2. (5 Points) Calculated document frequency (DF) in a  distributed fashion. 
3. (20 Points) Partitioned the posting lists and wrote them to disk in parallel.
4. (20 Points) Constructed the links graph and calculated PageRank on it. 
All of these will be based on a small subset of articles from one wiki dump file.

In GCP (35 Points):

1. (10 Points) Successfully started a Dataproc cluster, setup cloud storage bucket, and properly initialized cluster nodes.
2. (10 Points) Created an inverted index for all of English Wikipedia. 
3. (10 Points) Successfully computed PageRank for the entire English Wikipedia.
4. (5 Points) Reporting: size of the data, size of index, time it took to build the index, time it took to compute PageRank, and the $ amount of credits it consumed.

Bonus (10 points) if you implement PageRank in pure PySpark, i.e. without using the GraphFrames package, AND manage to complete 10 iterations of your algorithm on the entire English Wikipedia in less than an hour. 







# Setup

## General imports

The `inverted_index_colab` import requires the `inverted_index_colab.py` file. You should upload the file and then run this cell.

In [2]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index_colab import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Installing, importing, and initializing PySpark


The following code installs PySpark and its dependencies in Colab. In addition, we install GraphFrames, which is a PySpark package for dealing with graphs in a distributed fashion. Colab notebooks run on a single machine so we will work in local mode, i.e. there is no cluster of machines and both the master and worker processes run on a single machine. This will help us debug and iron out the code we will use in the second half of this assignment on an actual cluster mode in GCP. 

The installation in the next cell should take about 1 minute in a fresh environment. Don't worry about going  over the 90 seconds limit for the assignment because in our testing enviroment all of these requirements will already be met. 

In [3]:
# These will already be installed in the testing environment so disregard the 
# amount of time (~1 minute) it takes to install. 
!pip install -q pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

     |████████████████████████████████| 281.4 MB 45 kB/s 
     |████████████████████████████████| 199 kB 56.7 MB/s 
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 20 not upgraded.
Need to get 36.6 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 124016 files and directories currently installed.)
Preparing to unpack .../openjdk-8-jre-headless_8u352-ga-1~18.04_amd64.deb ...
Unpacking op

In [4]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [5]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
conf.set("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [6]:
spark

## Copy some wiki data

As mentioned above, all wiki dumps were preprocessed and placed in a shared google storage bucket. To access the files in Colab, you will first have to authenticate with google storage, and then copy the data to your local environment. 

**IMPORTANT NOTE**

Before you start working with the wiki data, you first have to go over the 
"Working with GCP.pdf" Provided to you under the gcp folder in the same zip file as this folder. In that pdf you will redeem your credits for the GCP and create your instance. You have to do this procedure because the files will be mounted to you through GCP process. 
Notice that you have to change the project_id variable in the second cell below. 

In GCP, the storage will already be mounted on the cluster machines and we will show you how to access it.


In [7]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
import signal

AUTH_TIMEOUT = 30

def handler(signum, frame):
  raise Exception("Authentication timeout!")

try:
  signal.signal(signal.SIGALRM, handler)
  signal.alarm(AUTH_TIMEOUT)
  auth.authenticate_user()
  signal.alarm(0)
except: 
  pass

In [8]:
# Copy one wikidumps files 
import os
from pathlib import Path
from google.colab import auth
## RENAME the project_id to yours project id from the project you created in GCP 
project_id = 'dependable-data-370310'
!gcloud config set project {project_id}

data_bucket_name = 'wikidata20210801_preprocessed'
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass  
except:
      !mkdir wikidumps
      !gsutil -u {project_id} cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/" 


Updated property [core/project].
Copying gs://wikidata20210801_preprocessed/multistream1_preprocessed.parquet...
| [1 files][316.7 MiB/316.7 MiB]                                                
Operation completed over 1 objects/316.7 MiB.                                    


# Processing wikipedia

Now that we completed the setup and have some data in our local environment, we are ready to process it using PySpark. 

## A 2-minute intro to PySpark

One of the fundamental data structures in spark is a **Resillient Distributed Dataset (RDD)**. It is an immutable distributed collections of objects that are partitioned across machines, and thus making parallel processing natural and easy. RDD's support two types of operations: *transformations* (e.g. selecting a subset of fields from each element or projecting each element from A->B), and *actions* (e.g. save to disk or `collect` elements into the master node's memory). Watch out for `collect` operations that send too much data to the master node, it will crash without a trace because of running out of memory. 

Spark employs **lazy-evaluation** (like many of us...), meaning that it accumulates things we ask it to do into an execution plan, but doesn't do any calculation until a result is actually needed. When a result is needed? when we save something to disk, ask spark to show the data, or use other actions. But make no mistakes, Spark is not a slacker. It employs lazy-evaluation because it allows optimizations that wouldn't be possible otherwise and it often saves users from unnecessary data reads, tranfers across machines, and other costly operations. As we will see in a bit, the RDD data structure lends itself to simple and efficient MapReduce operations.

Let's look at our data before transforming it to RDD.

In [38]:
from pathlib import Path 
import os

try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"

parquetFile = spark.read.parquet(path)
# take the 'text' and 'id' or the first 1000 rows and create an RDD from it
listi=[2310146, 970755, 17668101, 964614, 16389, 51759111, 1372169, 321546, 49051658, 54671372, 651278, 5892113, 456722, 16635934, 1380383, 55679006, 28985374, 25935906, 15357987, 19431459, 5144613, 38950, 11147298, 1857574, 5027882, 11028525, 1185840, 48490547, 55836725, 3514423, 6428730, 9254970, 42799166, 55947330, 878659, 2116, 2117, 63809606, 1417287, 1996872, 50602056, 9306179, 1151047, 22423628, 1790029, 33255495, 12789839, 886856, 22736969, 16842834, 9513043, 56107088, 11665493, 37646421, 1321047, 18952271, 43343961, 9988187, 2721889, 2082914, 1390689, 252008, 4253801, 2461806, 489585, 51546226, 42555506, 3762294, 52209782, 1017976, 54046846, 194687, 49016960, 979072, 61603971, 1386629, 2199688, 233609, 43030666, 2578570, 43794572, 30861, 31326350, 19988623, 3639440, 168079, 43794574, 48640150, 188572, 53215263, 17596651, 56494240, 1646753, 41726116, 60739751, 184488, 36827305, 57374888, 10328235, 1511596, 192685, 49029294, 1626287, 2883760, 5222577, 4294832, 42528947, 16615604, 28139692, 56170677, 50276542, 32961, 4743361, 7811267, 63520964, 14533, 53446, 231623, 2029766, 37468361, 23830729, 15218891, 3356874, 1079500, 8389, 1546447, 18640, 2367697, 383180, 50399439, 63484108, 65595607, 30687447, 16128216, 473309, 30942, 23189729, 8802530, 2275, 1566948, 7932132, 41648358, 12521, 9998569, 2732267, 663788, 55935213, 6887661, 13476079, 563439, 51765484, 16285933, 3574003, 14580, 56312051, 56312054, 67608822, 17742072, 1186041, 19159283, 3647739, 56961277, 727293, 7594242, 43561218, 14597, 884998, 14598, 26257672, 8460, 27857167, 48920848, 56289553, 35549457, 43281, 45332, 43284, 55339286, 66040086, 8728856, 4925720, 66040084, 84252, 24754461, 803102, 41244, 4335905, 4561188, 248101, 1962277, 55339303, 12523816, 6832430, 5591344, 25258288, 56398129, 3045683, 233780, 6201653, 35281209, 11125049, 59707, 88380, 442684, 45271353, 717119, 67987778, 1186115, 50164035, 32516422, 54802759, 52793670, 2377, 8522, 45111627, 63539530, 48795986, 64999764, 2685269, 2015573, 57041239, 57688, 14942276, 29129051, 36794719, 26976, 893280, 624998, 29868391, 56142183, 5024105, 37325161, 24230253, 8655214, 2353519, 62976368, 76143, 6003061, 38889846, 50762105, 61073786, 6826364, 2331004, 1567101, 215424, 315776, 68213121, 60457349, 6236554, 63656330, 30876044, 4041101, 784781, 2150801, 41699729, 410007, 1556887, 14745, 45470, 61014433, 57762, 44442017, 57932194, 17158563, 55822753, 295335, 838057, 5145001, 63656361, 63656365, 7432624, 65708464, 31429041, 4531, 373172, 53897655, 17842616, 9972157, 19698110, 1333695, 34374079, 18176448, 27148738, 65212863, 55708102, 3928523, 53707, 2535885, 608718, 3736012, 52709838, 22989, 31824340, 20529621, 164311, 20566488, 18784729, 1683930, 53836251, 98778, 1731036, 1151454, 145891, 1843684, 22888933, 61805032, 20611562, 3602925, 27119, 1356272, 6511088, 17349106, 59892, 4723188, 9511414, 33350134, 731640, 34075129, 172538, 43250171, 4538366, 64999924, 655861, 20892159, 1071613, 1028614, 27300359, 8716, 4827661, 3848717, 27007503, 53871120, 20810258, 72214, 18967, 53350936, 17418777, 1221144, 254496, 5497377, 8581665, 743971, 1045027, 37412, 61592102, 49127974, 60410401, 2996777, 36581929, 51046955, 47660, 56480301, 11577897, 37497391, 453166, 383537, 682549, 43600438, 604727, 62528055, 19004, 42072639, 2929216, 49728, 2761281, 33825347, 141896, 40925771, 53837, 17719886, 23353937, 840273, 35072597, 4090453, 1466966, 25959000, 731735, 223834, 6650456, 6996576, 150113, 39000674, 39639653, 275047, 23290471, 6281833, 1227367, 67830379, 27226732, 36971117, 152176, 2372209, 19058, 34519668, 127604, 387703, 3060346, 24724090, 8258172, 199292, 1661566, 1131136, 764545, 309890, 44311171, 1276547, 47956615, 60637832, 32051848, 41337483, 19049100, 8821389, 38156944, 65073808, 33018516, 14473878, 408215, 56885915, 27769500, 1751707, 12343966, 37534, 3060382, 68188835, 48530084, 7322279, 31885991, 8608425, 43033258, 1649321, 65741484, 47789, 21173, 522934, 27318, 678583, 45756, 25164479, 8559295, 53078721, 2327234, 3508935, 932553, 1632972, 49128142, 9833167, 1370831, 1673945, 52261594, 494299, 22481627, 27306717, 3398365, 31453, 41974496, 35646178, 4604645, 13824744, 17296107, 44712684, 12020461, 277231, 10992, 60283633, 36029170, 64467696, 5516020, 30640885, 1897206, 10998, 19189, 5653238, 52183794, 27837170, 65004286, 6910, 7133952, 24963841, 37249793, 1159939, 1104639, 50137861, 39683, 1581831, 33700618, 6109962, 66603787, 3799826, 62026514, 51237650, 8866584, 41974555, 23324, 62372638, 60744481, 4387617, 47905, 4506407, 408360, 24920873, 49134382, 48778030, 1833777, 1424178, 20790067, 43826, 53487, 24470328, 19261, 47227709, 1108803, 17521476, 402244, 9399111, 20185928, 13327177, 400199, 265033, 844, 30120784, 3367760, 10179411, 4870997, 660310, 66391, 61651800, 856, 21211994, 6271835, 20587357, 80735, 16360289, 265059, 35666788, 11447140, 82789, 37321573, 6722408, 5624681, 42881894, 60463979, 2786155, 3548013, 65819511, 37756, 1031040, 105344, 53615490, 3482503, 11938696, 1020809, 11486091, 5964683, 27202445, 8596369, 7555986, 2763667, 60754840, 10865561, 31552410, 553883, 19356, 2593693, 19357, 289691, 2165666, 56226, 713636, 16407460, 62417830, 15271, 54258598, 11164587, 10822574, 301999, 42730418, 846772, 33407925, 211895, 54653881, 9146, 17826747, 320443, 19390, 2198463, 494528, 33727, 453584, 1971153, 60611538, 168915, 168917, 19938267, 1686492, 36070366, 1958879, 168927, 961505, 4776930, 15457257, 1149933, 3785715, 314356, 39379960, 615418, 29324283, 791546, 19266557, 40659966, 67838974, 62741501, 3015678, 66556, 28541957, 28572685, 1088531, 57068567, 607255, 46105, 56132631, 17359901, 18168862, 48876576, 8438818, 7771171, 52247588, 22230053, 66405413, 66423851, 140332, 7414829, 25918508, 33672235, 1625137, 1041458, 1039412, 18691124, 205878, 16966712, 53431353, 49099835, 74813, 48188481, 61269058, 27345986, 1414212, 1879108, 3775558, 3138, 65967176, 261193, 42947658, 3556431, 45280337, 1371219, 21179478, 44254295, 14109784, 28890200, 463961, 683094, 1166428, 66174045, 28396636, 60601430, 52316, 32472154, 4226137, 41677925, 2376810, 11370, 42163310, 23133297, 15474, 31505523, 56435, 39302261, 302201, 1623162, 38321273, 3097723, 33336442, 244862, 4093054, 269441, 1291393, 11287682, 4416646, 57312392, 8778890, 36916362, 30860428, 29838, 6319249, 9299090, 52356241, 63732884, 21652, 17659030, 57390230, 3781784, 1936537, 26731675, 4512923, 44975261, 13190302, 1481886, 1385632, 1531043, 14040227, 63499429, 5864614, 58209447, 35804330, 64543917, 375986, 32695480, 66299065, 971961, 22291643, 21694, 24534207, 3890370, 27848, 2536648, 46734540, 3288269, 251087, 52184272, 400593, 4576465, 109780, 27862, 19074264, 30276826, 58586, 5033181, 14020881, 64928991, 15295713, 7955681, 57240806, 22156522, 57918704, 14003441, 18550003, 51430647, 57918711, 417018, 66811, 32670973, 1664254, 175357, 9825536, 49749249, 60534017, 39218436, 611589, 832774, 345354, 15628, 3099917, 34061, 32707853, 5391, 62166289, 548115, 34069, 68117784, 13593, 634139, 2694428, 277790, 5407, 45485344, 1183979, 13603, 273700, 21175589, 20657443, 13607, 2243880, 43603241, 316711, 273707, 234796, 1039663, 5801264, 1908019, 1817908, 19561784, 75065, 21347643, 77118, 1344, 99648, 60360004, 1357127, 37764426, 183628, 6614349, 3591502, 1248592, 175440, 10833234, 6622547, 2866516, 4478297, 3757402, 5612891, 10710364, 630109, 8734046, 14325087, 63280480, 20866399, 25490788, 2020710, 30057, 5580137, 5334377, 33463661, 61715824, 13681, 20448627, 42931572, 2927992, 8394105, 15739, 39740796, 47048067, 3706246, 19283335, 45690249, 8318345, 400777, 5721484, 22226313, 4208015, 9110929, 26639763, 34753948, 667037, 3548574, 22912415, 8521120, 8177057, 67110306, 33131935, 38310, 3841447, 230828, 25130414, 175537, 28155315, 1287604, 5201333, 292279, 19537336, 2526649, 5561, 38327, 7564733, 2495940, 14501317, 3519942, 9649607, 1500618, 73165, 4576717, 619983, 62929, 2614738, 67730903, 23891416, 6890967, 13532634, 19064282, 33310173, 35077599, 1074657, 51895777, 13594085, 146918, 1455590, 19920359, 47762921, 19619306, 20063724, 18863597, 67055, 19760623, 31403505, 255475, 31188467, 1631732, 3915251, 60920, 83449, 19600890, 43875835, 2020857, 18998781, 22707918, 52342272, 8351234, 19006979, 2672131, 962053, 482824, 8957449, 15881, 65175052, 54459918, 49233423, 64695824, 224785, 48317971, 10589717, 14900757, 2045465, 12054042, 23154203, 720414, 67520032, 51562019, 50066979, 656933, 64413225, 4939306, 390698, 4748844, 73262, 55490096, 159284, 101942, 31704630, 6032951, 56000054, 34361, 44240443, 19293758, 13890, 2913859, 1140293, 4939334, 1295947, 345676, 67147, 32327247, 73298, 41283158, 79449, 50191962, 48732, 5285468, 53151326, 10671710, 360030, 55819873, 18300514, 7489122, 7780, 38936168, 13805160, 15468138, 33508970, 65601132, 46728817, 1793651, 2627188, 44379765, 9845, 63946361, 6332026, 2580091, 3708, 17997437, 40573, 49131135, 312963, 60333700, 29384326, 63641225, 4939408, 63121, 1492625, 3440273, 5676692, 66422422, 10765975, 7872152, 341658, 36511, 4726434, 366244, 48461477, 4378282, 17774253, 323246, 32595633, 296627, 4939444, 22216378, 509628, 407234, 2322115, 49643204, 36439749, 52958915, 65228, 63534797, 35368654, 14343887, 1472206, 1005263, 15183570, 4939471, 612052, 52780757, 220886, 57147095, 39812824, 69336, 11996885, 390875, 38416091, 1853149, 2236126, 11884255, 66111204, 466664, 3260137, 200427, 56078060, 4939501, 909036, 7329519, 5902061, 42737, 52227830, 5078775, 407288, 38579961, 3038969, 1179384, 4939519, 43771647, 49387265, 60616450, 3315459, 54251265, 300805, 57098, 63031051, 14094, 41185040, 11947794, 26924822, 23619350, 26457880, 11249433, 59629338, 61972257, 22309, 32950054, 25913130, 20647724, 63180590, 56921904, 43345713, 22777652, 47863605, 67450679, 50716473, 6825785, 57146, 4554556, 1230653, 64051004, 26091326, 53358397, 915258, 63616836, 2828101, 1072968, 21821257, 22611786, 50865995, 1455948, 1394509, 8046414, 22114132, 5451605, 43407192, 56178521, 311130, 13799261, 3608414, 1632099, 485220, 124779, 52682605, 4718446, 47720307, 12095348, 7851893, 42870, 42871, 65464184, 36323189, 264058, 280437, 1242998, 18863997, 16760693, 47744894, 20481920, 32352129, 24452, 11263877, 636806, 1326984, 57141131, 649100, 1392524, 135063, 13602714, 29681566, 13834142, 5212064, 1527716, 60952488, 26537, 17362858, 645042, 57069491, 26814387, 2891701, 22933429, 2418612, 42311608, 5533631, 42680256, 40030145, 32706, 49170369, 24997830, 40542151, 927688, 60774345, 21991369, 1335238, 52228042, 1910732, 1998, 200646, 12240, 913362, 30846934, 94167, 128987, 22190045, 22144990, 1298502, 18402, 13365219, 167906, 46208997, 20455, 5863400, 51714024, 3549164, 31406060, 2891758, 3276784, 32325617, 14825456, 42994, 3901428, 3602421, 296950, 180211, 14320, 5818361, 10235, 20948989, 1148926, 16383]
filterIds = parquetFile[parquetFile["id"].isin(listi)]
doc_text_pairs = filterIds.select("text", "id").rdd
doc_title_pairs = filterIds.select("title", "id").rdd


doc_anchor_pairs = filterIds.select("anchor_text").rdd

def map_func(doc_anchor):
  edge_set = []
  for id, anchor in doc_anchor[0]:
    if (id, anchor) not in edge_set:
      edge_set.append((id, anchor))
  return edge_set

doc_anchor_pairs_new = doc_anchor_pairs.flatMap(map_func).distinct().groupByKey().mapValues(list).map(lambda x:(" ".join(x[1]),x[0]))
filterIds.show()

+----+--------------------+--------------------+--------------------+
|  id|               title|                text|         anchor_text|
+----+--------------------+--------------------+--------------------+
| 844|           Amsterdam|'''Amsterdam''' (...|[{169683, Capital...|
| 856|          Apple Inc.|'''Apple Inc.''' ...|[{40780688, Apple...|
|1344|             Apple I|The '''Apple Comp...|[{1115478, Comput...|
|1998|       Austin, Texas|'''Austin''' (, )...|[{255627, State c...|
|2116|     Apple II series|alt=|thumb|300x30...|[{25122906, home ...|
|2117|           Apple III|The '''Apple III'...|[{856, Apple Comp...|
|2275|            Apple II|thumb|240px|An Ap...|[{95171, Panasoni...|
|2377|      Andhra Pradesh|'''Andhra Pradesh...|[{375986, State},...|
|3138|             Atlanta|'''Atlanta''' () ...|[{255627, State c...|
|3708|            Brussels|'''Brussels''' ( ...|[{151664, Region ...|
|4531|    Bipolar disorder|'''Bipolar disord...|[{18973869, Psych...|
|5391|              

## Word counts

### Term frequency

**YOUR TASK (10 POINTS):** Complete the implementation of `word_count`, which calculates the term-frequency (tf) for each word in a single document.  map function. The functions gets the text and the wiki id, calculates the tf and returns list of tuples in the form of (token, (wiki_id, tf)).

In [39]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords =["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
all_stopwords = english_stopwords.union(corpus_stopwords)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)

def tokenize(text):
    """
    This function aims in tokenize a text into a list of tokens. Moreover, it filter stopwords.
    
    Parameters:
    -----------
    text: string , represting the text to tokenize.    
    
    Returns:
    -----------
    list of tokens (e.g., list of tokens).
    """
    list_of_tokens =  [token.group() for token in RE_WORD.finditer(text.lower()) if token.group() not in all_stopwords]    
    return list_of_tokens

def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  # YOUR CODE HERE
  return([(k,(id,v)) for k,v in Counter(tokens).items() if k not in all_stopwords and v>0])

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  return sorted(unsorted_pl, key=lambda tup: tup[0])

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  return postings.map(lambda x:(x[0],len(x[1])))

  NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  # YOUR CODE HERE
  dict1 = postings.map(lambda token: (token2bucket_id(token[0]), token)).groupByKey()
  return dict1.map(InvertedIndex.write_a_posting_list)

# merge the posting locations into a single dict and run more tests
def merge(postings):
  super_posting_locs = defaultdict(list)
  for posting_loc in postings:
    for k, v in posting_loc.items():
      super_posting_locs[k].extend(v)
  return super_posting_locs

def calc_dl(tokens,doc_id):
  return (doc_id,len(tokens))

def calc_total_term(postings):
  return postings.mapValues(helpFunc).collectAsMap()

def helpFunc(posting):
  count=0
  for tup in posting:
    count+=tup[1]
  return count

In [31]:
min_title=0
min_body=10
min_anchor=0
!mkdir body_index title_index anchor_index

mkdir: cannot create directory ‘body_index’: File exists
mkdir: cannot create directory ‘title_index’: File exists
mkdir: cannot create directory ‘anchor_index’: File exists


In [40]:
#inverted index body
index_body=InvertedIndex()
word_count_bodys = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
body_dl=doc_text_pairs.map(lambda x: calc_dl(tokenize(x[0]),x[1])).collectAsMap()
index_body.DL=body_dl

posting_lists_body = word_count_bodys.groupByKey().mapValues(reduce_word_counts)
filter_bodys = posting_lists_body.filter(lambda x: len(x[1])>min_body)
index_body.term_total=calc_total_term(filter_bodys)
index_body.df=calculate_df(filter_bodys).collectAsMap()

posting_locs_body = partition_postings_and_write(filter_bodys).collect()
posting_locs_body=merge(posting_locs_body)
index_body.posting_locs=posting_locs_body
index_body.write_index('body_index', 'index_body')

#inverted index title
index_title=InvertedIndex()
word_count_titles = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))
title_dl=doc_title_pairs.map(lambda x: calc_dl(tokenize(x[0]),x[1])).collectAsMap()
index_title.DL=title_dl

posting_lists_title = word_count_titles.groupByKey().mapValues(reduce_word_counts)
filter_titles = posting_lists_title.filter(lambda x: len(x[1])>min_title)
index_title.term_total=calc_total_term(filter_titles)
index_title.df=calculate_df(filter_titles).collectAsMap()

posting_locs_title = partition_postings_and_write(filter_titles).collect()
posting_locs_title=merge(posting_locs_title)
index_title.posting_locs=posting_locs_title
index_title.write_index('title_index', 'index_title')

#inverted index anchor
index_anchor=InvertedIndex()
word_count_anchors = doc_anchor_pairs_new.flatMap(lambda x: word_count(x[0], x[1]))
index_dl=doc_anchor_pairs_new.map(lambda x: calc_dl(tokenize(x[0]),x[1])).collectAsMap()
index_anchor.DL=index_dl

posting_lists_anchor = word_count_anchors.groupByKey().mapValues(reduce_word_counts)
filter_anchors = posting_lists_anchor.filter(lambda x: len(x[1])>min_anchor)
index_anchor.term_total=calc_total_term(filter_anchors)
index_anchor.df=calculate_df(filter_anchors).collectAsMap()

posting_locs_anchor = partition_postings_and_write(filter_anchors).collect()
posting_locs_anchor=merge(posting_locs_anchor)
index_anchor.posting_locs=posting_locs_anchor
index_anchor.write_index('anchor_index', 'index_anchor')